# Benchmarking RBF Kernel Performance

This notebook compares the execution time of the RBF kernel function in scikit-learn and sklearnex. The goal is to evaluate the performance improvements using the oneDAL-optimized implementation.

### Methodology
- We generate random matrices of different row sizes (2, 5, 10, 100, 1000, 10000) with 3 columns.
- We measure the execution time of the RBF kernel function in scikit-learn and sklearnex.
- We compare the results for accuracy and compute the speedup factor.

### Setup & Environment

In [1]:
import platform
import psutil
import sklearn
import numpy as np
import pandas as pd

print("### System Information ###")
print(f"OS: {platform.system()} {platform.release()} ({platform.version()})")
print("Windows Edition: Windows 11 Enterprise")
print(f"Processor: {platform.processor()}")
print(f"CPU Cores: {psutil.cpu_count(logical=False)} (Physical), {psutil.cpu_count(logical=True)} (Logical)")
print(f"RAM: {round(psutil.virtual_memory().total / (1024**3), 2)} GB")
print("Graphics Card: Intel(R) Iris(R) Xe Graphics")

print("\n### Library Versions ###")
print(f"Python Version: {platform.python_version()}")
print(f"scikit-learn Version: {sklearn.__version__}")
print(f"NumPy Version: {np.__version__}")
print(f"Pandas Version: {pd.__version__}")

### System Information ###
OS: Windows 10 (10.0.22631)
Windows Edition: Windows 11 Enterprise
Processor: Intel64 Family 6 Model 154 Stepping 3, GenuineIntel
CPU Cores: 12 (Physical), 16 (Logical)
RAM: 63.64 GB
Graphics Card: Intel(R) Iris(R) Xe Graphics

### Library Versions ###
Python Version: 3.10.11
scikit-learn Version: 1.6.1
NumPy Version: 1.26.4
Pandas Version: 2.1.3


In [2]:
from timeit import default_timer as timer
from sklearn.metrics.pairwise import rbf_kernel as sklearn_rbf_kernel
from sklearnex.metrics.pairwise import rbf_kernel as sklearnex_rbf_kernel
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# a list of different sizes of data to be tested
row_sizes = [2, 5, 10, 100, 1000, 10000]
col_size = 3

results = []

for row in row_sizes:
    X = np.random.rand(row, col_size)
    Y = np.random.rand(row, col_size)

    for i in range(10):
        start = timer()
        sklearn_result = sklearn_rbf_kernel(X, Y)
        sklearn_time = timer() - start

        start = timer()
        sklearnex_result = sklearnex_rbf_kernel(X, Y)
        sklearnex_time = timer() - start

        results.append({
            "# of row": row,
            "sklearn_result": sklearn_result,
            "sklearnex_result": sklearnex_result,
            "sklearn_time": sklearn_time,
            "sklearnex_time": sklearnex_time
        })

### Results

The results are stored in a dataframe for easier interpretation and analysis.

- The `result_match` column verifies if both implementations produce identical results.
- The `speedup` column shows how much faster sklearnex is compared to scikit-learn.

In [3]:
import pandas as pd

df = pd.DataFrame(results)

# Compare the results element-wise and aggregate the results
df["result_match"] = df.apply(lambda row: np.allclose(row["sklearn_result"], row["sklearnex_result"]), axis=1)
df.drop(columns=["sklearn_result", "sklearnex_result"], inplace=True)

# Calculate the speedup
df["speedup"] = df["sklearn_time"] / df["sklearnex_time"]
df["speedup"] = df["speedup"].apply(lambda x: round(x, 2))

df

,# of row,sklearn_time,sklearnex_time,result_match,speedup
0,2,0.000575,0.002887,True,0.20
1,2,0.000702,0.000182,True,3.86
2,2,0.000532,0.000176,True,3.03
3,2,0.000510,0.000155,True,3.29
4,2,0.000506,0.000150,True,3.38
5,2,0.000477,0.000146,True,3.27
6,2,0.000470,0.000144,True,3.28
7,2,0.000471,0.000143,True,3.29
8,2,0.000468,0.000143,True,3.27
9,2,0.000487,0.000148,True,3.30


In [4]:
# Calculate the number of times the results match and 
# the average speedup for each row size
df_avg = df.groupby("# of row").agg({"result_match": "sum", "speedup": "mean"}).reset_index()
df_avg.columns = ["Row Size", "Result Match Count", "Average Speedup"]

df_avg

,Row Size,Result Match Count,Average Speedup
0,2,10,3.017
1,5,10,2.969
2,10,10,3.103
3,100,10,1.968
4,1000,10,6.358
5,10000,10,9.805


## Summary & Conclusion

- The results indicate that sklearnex consistently produces the same results as scikit-learn (`result_match = True`).
- Performance improvements vary depending on input size, with a notable speedup as the number of rows increases.
- For large datasets (e.g., 10,000 rows), sklearnex achieves up to a **10x speedup** compared to scikit-learn.
- However, for smaller datasets (e.g., row size = 2), sklearnex sometimes shows **slower performance**.